# Revisiting the Strategies in Federated Learning

This notebook contains the solution to the two exercises proposed in the practical Unit 5 of the Machine Learning II course. 

## Custom functions

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from typing import List, Dict, Optional, Tuple, Union
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
import flwr as fl
import numpy as np
from flwr.common import EvaluateRes, FitRes, Scalar
from flwr.common import Metrics
from flwr.server.client_proxy import ClientProxy

def generate_ann(lr: float = 5e-4):
    model = tf.keras.Sequential([
        Conv2D(32, 5, activation='relu', input_shape=(32, 32, 3)),
        Conv2D(32, 5, activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, 3, activation='relu'),
        Conv2D(64, 3, activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=['accuracy']
    )
    return model

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s

def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 'num_clients' partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    
    return train_ds, val_ds, test_ds

def get_parameters(net) -> List[np.array]:
    return net.get_weights()

def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net

def train(net, trainloader, epochs: int = 10):
    net.fit(trainloader[0], trainloader[1], epochs=epochs, batch_size=100)
    return net


def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        print('this is the config', config)
        self.net = train(self.net, self.trainloader)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, Client {self.cid} accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    

NUM_CLIENTS = 5
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

def client_fn(cid) -> FlowerClient:
    net = generate_ann()
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

def fit_config(server_round: int):
    return dict(server_round=server_round, local_epochs=3)

model = generate_ann()
params = get_parameters(model)
del model 

/home/ana/Documents/ml2-labs/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


## Architectures

We tested an improved architecture (see `generate_ann()` function) and launch 4 experiments with different fractions fits ($p= [0.1, 0.3, 0.6, 1.0]$). The results obtained were:

- $p=0.1$: 48%.
- $p=0.3$: 50.5%.
- $p=0.6$: 52.5%.
- $p=1$: 53.25%.

We fixed the number of clients to `NUM_CLIENTS=5` to allow each client to have enough images to train with. We increased the number of rounds to propagate the distributed dataset to `NUM_ROUNDS=10`. The convolutional architecture clearly increases the performance from the original baseline (which only uses dense layers) and increasing the `fraction_fit` parameter results in a slight improvement of the validation accuracy. This behavior is mainly due to the number of instances that are being used to train each round of the system. By using $p=0.1$ we are only allowing the system to use 10% of the image data to learn classification visual features. When increasing the value of $p$ we see a performance boost.

In [6]:
NUM_ROUNDS = 10
FRACTION_FIT = 0.1

strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT, 
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
    min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
    on_fit_config_fn=fit_config,
    evaluate_metrics_aggregation_fn=weighted_average

)

# Start simulation
res = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2024-03-14 19:39:41,183 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-03-14 19:39:44,499	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 19:39:45,344 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:10.68.58.17': 1.0, 'memory': 30465422132.0, 'object_store_memory': 15232711065.0, 'CPU': 20.0, 'accelerator_type:G': 1.0, 'GPU': 2.0}
INFO flwr 2024-03-14 19:39:45,345 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 19:39:45,345 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 19:39:45,346 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 19:39:45,356 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 

(DefaultActor pid=3389325) [Client 0] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3389326) this is the config {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3389329) Epoch 1/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 15s 920ms/step - accuracy: 0.1500 - loss: 2.2951
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1361 - loss: 2.3012  
 5/18 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1226 - loss: 2.2977 
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1245 - loss: 2.3014
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1300 - loss: 2.2995  
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1235 - loss: 2.2986
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.2344 - loss: 2.0954 
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.2096 - loss: 2.2335
(DefaultActor pid=3389327) [Client 3] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 4x across cluster]
(DefaultActor pid=3389327) this is the config {'server_round': 1, 'l

DEBUG flwr 2024-03-14 19:39:58,499 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-03-14 19:39:58,503 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 19:39:58,504 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3389327) [Client 3] evaluate, config: {}
(DefaultActor pid=3389327) [Client 3] loss:1.752123475074768, Client 3 accuracy:0.35499998927116394
(DefaultActor pid=3389327) Epoch 10/10 [repeated 25x across cluster]


DEBUG flwr 2024-03-14 19:39:59,413 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:39:59,414 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-03-14 19:41:28,347 | server.py:187 | evaluate_round 9 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:41:28,348 | server.py:222 | fit_round 10: strategy sampled 5 clients (out of 5)


(DefaultActor pid=3389329) [Client 3] fit, config: {'server_round': 10, 'local_epochs': 3}
(DefaultActor pid=3389329) this is the config {'server_round': 10, 'local_epochs': 3}
 4/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6090 - loss: 1.1371  
(DefaultActor pid=3389326) Epoch 2/10 [repeated 17x across cluster]
 1/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.6200 - loss: 1.1045 [repeated 8x across cluster]
 4/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7531 - loss: 0.7349 [repeated 8x across cluster]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7499 - loss: 0.7655 [repeated 39x across cluster]
15/18 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7569 - loss: 0.7552 [repeated 26x across cluster]
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6483 - loss: 1.0527
(DefaultActor pid=3389329) [Client 0] evaluate, config: {}
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.7098 - loss: 0.8592 [repeated 3x across cluster]
(DefaultActor pid=3389329) [C

DEBUG flwr 2024-03-14 19:41:38,695 | server.py:236 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-03-14 19:41:38,699 | server.py:173 | evaluate_round 10: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3389328) [Client 4] evaluate, config: {}
(DefaultActor pid=3389328) [Client 4] loss:1.5812214612960815, Client 4 accuracy:0.5149999856948853


DEBUG flwr 2024-03-14 19:41:39,387 | server.py:187 | evaluate_round 10 received 2 results and 0 failures
INFO flwr 2024-03-14 19:41:39,387 | server.py:153 | FL finished in 114.02943027898436
INFO flwr 2024-03-14 19:41:39,388 | app.py:226 | app_fit: losses_distributed [(1, 1.7675231099128723), (2, 1.460463285446167), (3, 1.4926090836524963), (4, 1.3456531167030334), (5, 1.4010659456253052), (6, 1.4185267686843872), (7, 1.3055131435394287), (8, 1.400951087474823), (9, 1.362947940826416), (10, 1.8049861788749695)]
INFO flwr 2024-03-14 19:41:39,388 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 19:41:39,388 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.3699999898672104), (2, 0.45000000298023224), (3, 0.4599999934434891), (4, 0.5100000202655792), (5, 0.5174999982118607), (6, 0.5075000077486038), (7, 0.5225000083446503), (8, 0.5475000143051147), (9, 0.5349999964237213), (10, 0.47999998927116394)]}
INFO flwr 2024-03-14 19:41:39,389 | app.py:229 | app

In [7]:
NUM_ROUNDS = 10
FRACTION_FIT = 0.3

strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT, 
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
    min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
    on_fit_config_fn=fit_config,
    evaluate_metrics_aggregation_fn=weighted_average

)

# Start simulation
res = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2024-03-14 19:41:39,399 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-03-14 19:41:42,650	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 19:41:43,523 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 15218146099.0, 'node:10.68.58.17': 1.0, 'node:__internal_head__': 1.0, 'GPU': 2.0, 'accelerator_type:G': 1.0, 'CPU': 20.0, 'memory': 30436292199.0}
INFO flwr 2024-03-14 19:41:43,524 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 19:41:43,525 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 19:41:43,525 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 19:41:43,536 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 

(DefaultActor pid=3403378) [Client 0] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3403378) this is the config {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3403378) Epoch 1/10
 4/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1342 - loss: 2.3002  
 1/18 ━━━━━━━━━━━━━━━━━━━━ 16s 971ms/step - accuracy: 0.1100 - loss: 2.2979
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1315 - loss: 2.2981
 5/18 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0921 - loss: 2.3059 
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1044 - loss: 2.2964  
11/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1291 - loss: 2.2969
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1772 - loss: 2.2453
 2/18 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.0800 - loss: 2.3017
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.1603 - loss: 2.2335
(DefaultActor pid=3403379) [Client 2] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 4x across clus

DEBUG flwr 2024-03-14 19:41:56,706 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-03-14 19:41:56,710 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 19:41:56,711 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3403379) [Client 2] evaluate, config: {}
(DefaultActor pid=3403379) Epoch 10/10 [repeated 24x across cluster]
(DefaultActor pid=3403379) [Client 2] loss:1.8390014171600342, Client 2 accuracy:0.3499999940395355


DEBUG flwr 2024-03-14 19:41:57,583 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:41:57,583 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-03-14 19:43:26,560 | server.py:187 | evaluate_round 9 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:43:26,561 | server.py:222 | fit_round 10: strategy sampled 5 clients (out of 5)


(DefaultActor pid=3403377) [Client 4] fit, config: {'server_round': 10, 'local_epochs': 3}
(DefaultActor pid=3403377) this is the config {'server_round': 10, 'local_epochs': 3}
 5/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6402 - loss: 1.0774  
(DefaultActor pid=3403378) Epoch 1/10 [repeated 18x across cluster]
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6656 - loss: 0.9539 [repeated 5x across cluster]
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.7500 - loss: 0.8013 [repeated 11x across cluster]
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7327 - loss: 0.7683 [repeated 31x across cluster]
 5/18 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6319 - loss: 1.0415
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8007 - loss: 0.6137 [repeated 40x across cluster]
(DefaultActor pid=3403377) [Client 0] evaluate, config: {}
(DefaultActor pid=3403377) [Client 0] loss:1.3065807819366455, Client 0 accuracy:0.5149999856948853
(DefaultActor pid=3403378) [Cli

DEBUG flwr 2024-03-14 19:43:36,949 | server.py:236 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-03-14 19:43:36,958 | server.py:173 | evaluate_round 10: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3403378) [Client 3] evaluate, config: {}
(DefaultActor pid=3403378) [Client 3] loss:1.628924012184143, Client 3 accuracy:0.5199999809265137


DEBUG flwr 2024-03-14 19:43:37,693 | server.py:187 | evaluate_round 10 received 2 results and 0 failures
INFO flwr 2024-03-14 19:43:37,693 | server.py:153 | FL finished in 114.15497612897889
INFO flwr 2024-03-14 19:43:37,695 | app.py:226 | app_fit: losses_distributed [(1, 1.8151773810386658), (2, 1.5237781405448914), (3, 1.512254774570465), (4, 1.374237060546875), (5, 1.4044163227081299), (6, 1.4962342381477356), (7, 1.5625017881393433), (8, 1.3260533809661865), (9, 1.379119634628296), (10, 1.8153515458106995)]
INFO flwr 2024-03-14 19:43:37,696 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 19:43:37,696 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.33249999582767487), (2, 0.42499999701976776), (3, 0.4650000035762787), (4, 0.5125000029802322), (5, 0.5475000143051147), (6, 0.510000005364418), (7, 0.48999999463558197), (8, 0.5649999976158142), (9, 0.5349999964237213), (10, 0.5049999952316284)]}
INFO flwr 2024-03-14 19:43:37,696 | app.py:229 | app

In [8]:
NUM_ROUNDS = 10
FRACTION_FIT = 0.6

strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT, 
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
    min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
    on_fit_config_fn=fit_config,
    evaluate_metrics_aggregation_fn=weighted_average

)

# Start simulation
res = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2024-03-14 19:43:37,705 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-03-14 19:43:41,039	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 19:43:41,884 | app.py:213 | Flower VCE: Ray initialized with resources: {'GPU': 2.0, 'accelerator_type:G': 1.0, 'memory': 30356491470.0, 'CPU': 20.0, 'node:__internal_head__': 1.0, 'node:10.68.58.17': 1.0, 'object_store_memory': 15178245734.0}
INFO flwr 2024-03-14 19:43:41,884 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 19:43:41,885 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 19:43:41,885 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 19:43:41,896 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5069 - loss: 1.9366  
(DefaultActor pid=3403381) [Client 2] loss:2.001779079437256, Client 2 accuracy:0.49000000953674316


(DefaultActor pid=3417378) [Client 0] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3417378) this is the config {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3417378) Epoch 1/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 15s 909ms/step - accuracy: 0.1300 - loss: 2.3035
 8/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1229 - loss: 2.2969 
12/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1197 - loss: 2.2964
16/18 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1178 - loss: 2.2951
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1165 - loss: 2.2943
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1000 - loss: 2.2582
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1345 - loss: 2.2404
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1250 - loss: 2.2946
 9/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1403 - loss: 2.2354
 2/18 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.1250 - loss: 2.3067
11/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - ac

DEBUG flwr 2024-03-14 19:43:55,055 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-03-14 19:43:55,059 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 19:43:55,060 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3417377) [Client 2] evaluate, config: {}
(DefaultActor pid=3417377) Epoch 10/10 [repeated 24x across cluster]


...

DEBUG flwr 2024-03-14 19:45:24,753 | server.py:187 | evaluate_round 9 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:45:24,754 | server.py:222 | fit_round 10: strategy sampled 5 clients (out of 5)


(DefaultActor pid=3417375) [Client 1] fit, config: {'server_round': 10, 'local_epochs': 3}
(DefaultActor pid=3417375) this is the config {'server_round': 10, 'local_epochs': 3}
 9/18 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6280 - loss: 1.0643 
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6274 - loss: 1.1339
(DefaultActor pid=3417375) Epoch 2/10 [repeated 17x across cluster]
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7900 - loss: 0.7115 [repeated 9x across cluster]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7795 - loss: 0.6639 [repeated 4x across cluster]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.7299 - loss: 0.7871 [repeated 27x across cluster]
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7780 - loss: 0.6790 [repeated 34x across cluster]
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8072 - loss: 0.5981 [repeated 6x across cluster]
(DefaultActor pid=3417375) [Client 3] evaluate, config: {}
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 

DEBUG flwr 2024-03-14 19:45:35,079 | server.py:236 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-03-14 19:45:35,083 | server.py:173 | evaluate_round 10: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3417378) [Client 2] evaluate, config: {}
(DefaultActor pid=3417378) [Client 2] loss:2.097696304321289, Client 2 accuracy:0.47999998927116394


DEBUG flwr 2024-03-14 19:45:35,775 | server.py:187 | evaluate_round 10 received 2 results and 0 failures
INFO flwr 2024-03-14 19:45:35,775 | server.py:153 | FL finished in 113.87713180098217
INFO flwr 2024-03-14 19:45:35,776 | app.py:226 | app_fit: losses_distributed [(1, 1.7902809381484985), (2, 1.4459488987922668), (3, 1.4834538102149963), (4, 1.3314933776855469), (5, 1.3786522150039673), (6, 1.4523050785064697), (7, 1.4489492774009705), (8, 1.590196132659912), (9, 1.455549418926239), (10, 1.880920946598053)]
INFO flwr 2024-03-14 19:45:35,776 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 19:45:35,776 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.3475000113248825), (2, 0.47749999165534973), (3, 0.46000000834465027), (4, 0.5250000059604645), (5, 0.48499999940395355), (6, 0.502499982714653), (7, 0.5074999928474426), (8, 0.5), (9, 0.5374999940395355), (10, 0.5249999910593033)]}
INFO flwr 2024-03-14 19:45:35,777 | app.py:229 | app_fit: losses_ce

In [15]:
NUM_ROUNDS = 8
FRACTION_FIT = 1

strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT, 
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
    min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
    on_fit_config_fn=fit_config,
    evaluate_metrics_aggregation_fn=weighted_average

)

# Start simulation
res = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2024-03-14 19:53:50,398 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=8, round_timeout=None)
2024-03-14 19:53:53,623	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 19:53:54,495 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'accelerator_type:G': 1.0, 'GPU': 2.0, 'object_store_memory': 15111106560.0, 'memory': 30222213120.0, 'node:10.68.58.17': 1.0, 'CPU': 20.0}
INFO flwr 2024-03-14 19:53:54,496 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 19:53:54,496 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 19:53:54,497 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 19:53:54,508 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 2

(DefaultActor pid=3459591) [Client 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3459591) this is the config {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=3459591) Epoch 1/10
 4/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1077 - loss: 2.3002  
 9/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1020 - loss: 2.3001
12/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1033 - loss: 2.2991
 1/18 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.1200 - loss: 2.2966
 4/18 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1267 - loss: 2.2942
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.1643 - loss: 2.1981
(DefaultActor pid=3459593) [Client 4] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 4x across cluster]
(DefaultActor pid=3459593) this is the config {'server_round': 1, 'local_epochs': 3} [repeated 4x across cluster]
(DefaultActor pid=3459591) Epoch 6/10 [repeated 25x across cluster]
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step -

DEBUG flwr 2024-03-14 19:54:07,654 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-03-14 19:54:07,662 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 19:54:07,662 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 5)


(DefaultActor pid=3459595) [Client 2] evaluate, config: {}
(DefaultActor pid=3459595) [Client 2] loss:1.7876964807510376, Client 2 accuracy:0.4050000011920929
(DefaultActor pid=3459595) Epoch 10/10 [repeated 24x across cluster]


DEBUG flwr 2024-03-14 19:54:08,565 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2024-03-14 19:54:08,566 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 5)


...

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7740 - loss: 0.6820 [repeated 100x across cluster]
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8300 - loss: 0.5617 [repeated 17x across cluster]
(DefaultActor pid=3459594) [Client 1] fit, config: {'server_round': 8, 'local_epochs': 3}
(DefaultActor pid=3459594) this is the config {'server_round': 8, 'local_epochs': 3}
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6083 - loss: 1.0871
 3/18 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6194 - loss: 1.2175
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7107 - loss: 0.8786 [repeated 2x across cluster]
(DefaultActor pid=3459594) Epoch 6/10 [repeated 17x across cluster]
(DefaultActor pid=3459594) [Client 0] evaluate, config: {}
 2/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.6500 - loss: 1.0195 [repeated 7x across cluster]
 7/18 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7515 - loss: 0.7775 [repeated 40x across cluster]
(DefaultActor pid=3459594) [Cli

DEBUG flwr 2024-03-14 19:55:25,621 | server.py:236 | fit_round 8 received 5 results and 0 failures
DEBUG flwr 2024-03-14 19:55:25,628 | server.py:173 | evaluate_round 8: strategy sampled 2 clients (out of 5)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7574 - loss: 0.7193 [repeated 119x across cluster]
 1/18 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.6300 - loss: 0.8736 [repeated 21x across cluster]
(DefaultActor pid=3459592) [Client 0] evaluate, config: {}
(DefaultActor pid=3459592) [Client 0] loss:1.3061422109603882, Client 0 accuracy:0.5550000071525574


DEBUG flwr 2024-03-14 19:55:26,416 | server.py:187 | evaluate_round 8 received 2 results and 0 failures
INFO flwr 2024-03-14 19:55:26,417 | server.py:153 | FL finished in 91.90661490400089
INFO flwr 2024-03-14 19:55:26,423 | app.py:226 | app_fit: losses_distributed [(1, 1.7327715158462524), (2, 1.5202109813690186), (3, 1.416843593120575), (4, 1.4801118969917297), (5, 1.432280719280243), (6, 1.3275156617164612), (7, 1.2617117166519165), (8, 1.54569411277771)]
INFO flwr 2024-03-14 19:55:26,423 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 19:55:26,424 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.4050000011920929), (2, 0.4625000059604645), (3, 0.48249998688697815), (4, 0.48250000178813934), (5, 0.4999999850988388), (6, 0.5200000107288361), (7, 0.5374999940395355), (8, 0.5324999988079071)]}
INFO flwr 2024-03-14 19:55:26,424 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-03-14 19:55:26,424 | app.py:230 | app_fit: metrics_centralized

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5240 - loss: 1.7698  
